# K-Fold CV for Boosting Methods Using SKLearn 

In [1]:
#Importing required libraries
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm
import wfdb
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd
import random
from scipy.stats import randint

In [2]:
import xgboost as xgb
from sklearn import metrics
from lightgbm import LGBMClassifier

import lightgbm
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 70 #display 70 dpi in Jupyter Notebook, may consider100 dpi 
plt.rcParams['savefig.dpi'] = 300 #define 300 dpi for saving figures

import seaborn as sns
## here are some settings 
sns.set_style('whitegrid')
sns.set(rc={"figure.dpi":70, 'savefig.dpi':300}) #defining dpi setting
sns.set_context('notebook')
sns.set_style("ticks")
from IPython.display import set_matplotlib_formats

### To load sklearn dataset
sklearn.datasets.load_breast_cancer(*, return_X_y=False, as_frame=False)[source]¶

Load and return the breast cancer wisconsin dataset (classification).

In [4]:
#data = load_breast_cancer(as_frame = True); 
#df = data.frame; 
#print(df.shape)
# In this block use the code we looked at on 6/30 as a reference to save our rr-intervals as a .
rlist = []
records = 'mit-bih-raw\RECORDS' # Replace the _______ with the name of the records file in your MIT-BIH data folder
with open(records) as rfile: #Then we open the file 
                             #The 'with' command only opens the file while we are in it. Automatically closes the file when we're not
    for record in rfile:  # Then we iterate through the lines in the file
        record = record[0:len(record)-1] # Remove any erronious new line characters at the end ('\n')
        rlist.append(record) # Then build an array with it
        

###### Step 1: Initialize all Arrays
             # Below, replace all of the ___ with the command that declares an array/list
             # hint: https://stackoverflow.com/questions/1514553/how-to-declare-an-array-in-python
samples = [] # will house the samples of all subjects
good_list = [] # will list the names of the subjects we successfully extracted
bad_list = [] # will house the names of the subjects we failed to extract
qrs = [] # will house the indices of R-Peaks for all subjects
atr_label = [] # will house the labels for each rhythm annotation for all subjects
atr_locs = [] # will house the locations corresponding to the rhythm annotation labels


###### Step 2: Extract Information
for x in tqdm(rlist): #this will iterate through te records that we found above
  
    try: # A try statement will run the except statement if for some reason the try commands fail
         # In this case I use the try statement because one of the subjects has no signal data causing failure
         # I then use bad_list and good_list so that all of the indices in rlist match with the arrays we initialized in Step 1, above
        ######################################################
            # Below find the wfdb function that will return the information that is described below 
        # Then replace _____ with the correct function call
        samp = wfdb.rdsamp('mit-bih-raw/'+x) # wfdb._____(file_location) will read the signal & header data and return a 2 value array
            # samp[0] - the signal data is the raw reading from the ecg. Each value is a sample taken.
            # samp[1] - the header data includes things about the signal data such as:
              # samples per section, denoted 'fs'
              # number of signals, denoted 'n_sig'
            
        ######################################################
        samples.append(samp) #add it to our array for all subject
            #What is our file extension that has the annotation we want? Find it here and replace _____ with it 
            #hint: READ THE VARIABLE NAMES!!!!
        qrs_tmp = wfdb.rdann('mit-bih-raw/'+x, extension='qrs') #extract the QRS Info
        qrs_locs = np.array(qrs_tmp.sample, dtype='int') #Get just the loccation of R-Peaks from the QRS Info
        qrs.append(qrs_locs) # Add to our array for all subjects
        
            #Do the same thing here
        atr = wfdb.rdann('mit-bih-raw/'+x,extension='atr') #extract the atr info which stores the rhythm type(s) over the whole signal
        atr_label.append(atr.aux_note) # aux_note stores the type of rhythm - main two are '(N' for normal and '(AFIB' for AFIB
        atr_locs.append(np.append(atr.sample, len(samp[0]))) #I add the length of the whole sample to the end for better visualization later
        
        good_list.append(x) # when all extraction is successful append the record name to good_list
    except Exception as exep:
        print(exep) # Alert the user of an exception
        bad_list.append(x) # add to the bad list

        

  0%|          | 0/25 [00:00<?, ?it/s]

sampto must be greater than sampfrom
sampto must be greater than sampfrom


100%|██████████| 25/25 [00:22<00:00,  1.10it/s]


In [5]:
features_df_list = []
for subject in good_list:
    data = pd.read_csv(f'subject-features-dataframes/subject{subject}.csv')
    data = data.assign(Subject = subject)
    #print(data)
    features_df_list.append(data)
#features_maindf = pd.concat(features_df_list, ignore_index = True)

In [6]:
#df.head(5)
# df.info()
# df.describe()
#features_df_list[0]
counter = 0
testing_arr = []
training_arr = []
for subject in features_df_list:
    if counter != 22:
        training_arr.append(subject)
    else:
        testing_arr.append(subject)
    counter += 1
    
print(len(training_arr))

22


In [7]:
features_maindf = pd.concat(features_df_list, ignore_index = True)
print(features_maindf.columns)

Index(['Unnamed: 0', 'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
       'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Trans_AppEn', 'Rhythm',
       'Subject'],
      dtype='object')


In [8]:
def confusion_calcs(confusion_matrix):
    temp_arr = confusion_matrix
    tn = temp_arr[0]
    fp = temp_arr[1]
    fn = temp_arr[2]
    tp = temp_arr[3]
    sensitivity = tp / (tp + fn)
    recall = sensitivity
    specificity = tn / (tn + fp)
    precision = tp / (tp + fp)
    accuracy = (tp+tn) / (tp+tn+fn+fp)
    f1 = 2*precision*recall/(precision+recall)
    return [accuracy, sensitivity, specificity, precision, f1]

# Extreme Gradient Boosting

In [9]:
# Short to Short



# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
# check xgboost version
import xgboost
import xgboost as xgb
from xgboost import XGBClassifier
print(xgboost.__version__)
from matplotlib import pyplot
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as metrics

#Implementing cross validation 
kf = KFold(n_splits=23, shuffle=False)#, random_state=1)
# Create the Extreme GBM
model = XGBClassifier(n_estimators=100, learning_rate = 0.1,use_label_encoder =False,verbose=None,
                max_depth = 5, eval_metric='logloss')
    
from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated


y_values = ['Rhythm']
#'StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
#       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
#       'AppEn', 'SampEn', 'IQR', 'Var', 'Rmean_Var', 'Mean_Absolute_Deviation'
#for df in features_df_list:
df = features_maindf


import statistics
    

all_features = ['StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
        'AppEn', 'SampEn',"Trans_AppEn"]
dataScores = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
        
for score_idx,temp in tqdm(enumerate(all_features)):    
    
    x_values = [temp]
    print(x_values)

    acc_score = [];
    Truth = [];
    Output = [];

    for idx, subject in enumerate(good_list):

        train_index = ~df['Subject'].str.contains(subject)
        test_index = df['Subject'].str.contains(subject)
        X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
        y_train , y_test = df.loc[train_index,y_values], df.loc[test_index,y_values]

    #     temp_model = XGBClassifier(learning_rate = 0.1, use_label_encoder =False,verbose=None, eval_metric='logloss')
    #     param_distributions = {'n_estimators': randint(105, 110),
    #                            'max_depth': randint(7,10)}
    #     search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
    #     search.fit(X_train, y_train)

    #     n_estimators_list.append(search.best_params_['n_estimators'])
    #     max_depth_list.append(search.best_params_['max_depth'])    

        #model = XGBClassifier(n_estimators=100, learning_rate=0.1,use_label_encoder=False,verbose=None, max_depth=5, eval_metric='logloss')
        model.fit(X_train, y_train)
        #model.fit(X_train,y_train)

        pred_values = model.predict(X_test)
        pred_prob = model.predict_proba(X_test)[:, 1]

        acc = accuracy_score(y_test, pred_values)
        acc_score.append(acc)

        Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
        Output.extend(pred_values); ## it is a list 

        #print(model.feature_importances_)
    #     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
    #                           index = (X_train.columns))
    #     Importance.sort_values(by = 'Importance', 
    #                        axis = 0, 
    #                        ascending = True).plot(kind = 'barh', 
    #                                               color = 'r')
    #     plt.xlabel('Variable Importance')
    #     plt.gca().legend_ = None
    #     plt.show()
        #plt.savefig('plot1.png')

        ##################################################
        #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
        #Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.
        try:
            print("roc_auc_score", roc_auc_score(y_test, pred_prob))
        except:
            print(f'ROC_AUC not able to be calculated for subject {subject}')
        # calculate the fpr and tpr for all thresholds of the classification
        #fpr, tpr, threshold = metrics.roc_curve(y_test, pred_prob)
        #roc_auc = metrics.auc(fpr, tpr)
        # method I: plt
        #import matplotlib.pyplot as plt
        #plt.title('Receiver Operating Characteristic')
        #plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
        #plt.legend(loc = 'lower right')
        #plt.plot([0, 1], [0, 1],'r--')
        #plt.xlim([-0.05, 1])
        #plt.ylim([0, 1.05])
        #plt.ylabel('True Positive Rate')
        #plt.xlabel('False Positive Rate')
        #plt.show()
        ##################################################

    elapsed = timeit.default_timer() - start_time #gives total computation time
    print("---Run time is %s seconds ---" % elapsed) #prints computation time
    print()

    print('Accuracy of each fold: \n {}'.format(acc_score))
    print()
    print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
    print()
    print('Std of accuracy : \n{}'.format(np.std(acc_score)))
    print(confusion_matrix(Truth, Output))
    print()
    print(classification_report(Truth, Output))
    print()
    print(acc_score)
    print()
    print(elapsed)
    
    temp_array = confusion_matrix(Truth, Output)
    joint = temp_array[0].tolist() + temp_array[1].tolist()
    temp_scores = confusion_calcs(joint)
    
    #dataScores[score_idx].append(all_features[score_idx])
    #dataScores[score_idx].append(acc_score)
    dataScores[score_idx].append(np.mean(acc_score))
    dataScores[score_idx].append(np.std(acc_score))
    
    for confusionScore in temp_scores:
        dataScores[score_idx].append(confusionScore)
    

0it [00:00, ?it/s]

1.5.1
['StoS']
roc_auc_score 0.9738059724209309
roc_auc_score 0.7813857832988267
roc_auc_score 0.9971437013690535
roc_auc_score 0.9803756225924783
roc_auc_score 0.9530519399211749
roc_auc_score 0.7303213500084607
roc_auc_score 0.5363209065520556
roc_auc_score 0.9944672131147541
roc_auc_score 0.8491329586566142
roc_auc_score 0.9813077004973461
roc_auc_score 0.7018919105307924
roc_auc_score 0.776483413132695
roc_auc_score 0.7821723162586817
ROC_AUC not able to be calculated for subject 07162
ROC_AUC not able to be calculated for subject 07859
roc_auc_score 0.968420787744924
roc_auc_score 0.9277637603962245
roc_auc_score 0.2696180722268594
roc_auc_score 0.6068568553901458
roc_auc_score 0.8346764476233625
roc_auc_score 0.8710001016880211
roc_auc_score 0.9727491039426523


1it [00:10, 10.44s/it]

roc_auc_score 0.9437591327375165
---Run time is 10.267369500000001 seconds ---

Accuracy of each fold: 
 [0.8483981693363845, 0.24178498985801217, 0.9861460957178841, 0.8920863309352518, 0.8359456635318704, 0.09776876267748479, 0.6202709014479215, 0.9904697072838666, 0.7399598393574297, 0.7276736493936052, 0.6311252268602541, 0.9849246231155779, 0.6063492063492063, 0.3159235668789809, 0.5645295587010825, 0.8817013735046522, 0.9129540781357094, 0.31217541834968265, 0.2569737954353339, 0.8290269378779549, 0.8387510692899914, 0.9648462021343377, 0.852431289640592]

Avg accuracy : 
0.6927050632962202

Std of accuracy : 
0.2669211118961723
[[15295  9448]
 [ 5524 14701]]

              precision    recall  f1-score   support

           0       0.73      0.62      0.67     24743
           1       0.61      0.73      0.66     20225

    accuracy                           0.67     44968
   macro avg       0.67      0.67      0.67     44968
weighted avg       0.68      0.67      0.67     44968

2it [00:21, 10.70s/it]

...

roc_auc_score 0.9987561748036287
---Run time is 274.47420050000005 seconds ---

Accuracy of each fold: 
 [0.8838672768878718, 0.9379310344827586, 0.9937027707808564, 0.6354916067146283, 0.9796238244514106, 0.9221095334685598, 0.8902382064455862, 0.988427501701838, 0.9252008032128514, 0.8787210584343991, 0.9800362976406534, 0.9633883704235463, 0.7315192743764173, 1.0, 0.2922564529558701, 0.844040762073549, 0.976010966415353, 0.9971148297749567, 0.6191885038038885, 0.9472237493128093, 0.9666381522668948, 0.935969868173258, 0.9446088794926004]

Avg accuracy : 
0.8797091184039374

Std of accuracy : 
0.16313739372799654
[[21551  3192]
 [ 2721 17504]]

              precision    recall  f1-score   support

           0       0.89      0.87      0.88     24743
           1       0.85      0.87      0.86     20225

    accuracy                           0.87     44968
   macro avg       0.87      0.87      0.87     44968
weighted avg       0.87      0.87      0.87     44968


[0.88386727688787

16it [04:52, 23.37s/it]

roc_auc_score 0.9993360978040373
---Run time is 292.7633978 seconds ---

Accuracy of each fold: 
 [0.8832951945080092, 0.9460446247464503, 0.993073047858942, 0.6199040767386091, 0.9817136886102403, 0.922920892494929, 0.9065857076132648, 0.9891082368958475, 0.9272088353413654, 0.8158765159867696, 0.9823049001814882, 0.9569274946159368, 0.7034013605442176, 1.0, 0.37635303913405493, 0.8852459016393442, 0.9677861549006168, 0.9976918638199653, 0.5718512256973796, 0.9538207806487081, 0.9820359281437125, 0.9146264908976773, 0.9682875264270613]

Avg accuracy : 
0.8802636298888953

Std of accuracy : 
0.15783498513557123
[[21115  3628]
 [ 2185 18040]]

              precision    recall  f1-score   support

           0       0.91      0.85      0.88     24743
           1       0.83      0.89      0.86     20225

    accuracy                           0.87     44968
   macro avg       0.87      0.87      0.87     44968
weighted avg       0.87      0.87      0.87     44968


[0.8832951945080092, 

17it [05:05, 17.97s/it]

roc_auc_score 0.9947942229778277
---Run time is 305.2756353 seconds ---

Accuracy of each fold: 
 [0.8770022883295194, 0.922920892494929, 0.9848866498740554, 0.7583932853717026, 0.9639498432601881, 0.8912778904665314, 0.8696870621205044, 0.9639210347174949, 0.9031124497991968, 0.7717750826901875, 0.9351179673321234, 0.9368269921033741, 0.6412698412698413, 0.9528662420382166, 0.6582014987510408, 0.9530350022153301, 0.9575051405071967, 0.8990190421234853, 0.4611158072696534, 0.9483232545354591, 0.9563729683490163, 0.9296924042686755, 0.973784355179704]

Avg accuracy : 
0.8743503041333662

Std of accuracy : 
0.12856620226571613
[[20484  4259]
 [ 1754 18471]]

              precision    recall  f1-score   support

           0       0.92      0.83      0.87     24743
           1       0.81      0.91      0.86     20225

    accuracy                           0.87     44968
   macro avg       0.87      0.87      0.87     44968
weighted avg       0.87      0.87      0.87     44968


[0.8770

In [10]:

    
export_df = pd.DataFrame(dataScores,columns=['Mean Acc.', 'SD of Acc.', 'Total Acc.', 'Sensitivity', 
                                             'Specificity','Precision','f1'])

export_df = export_df.set_index(pd.Index(['StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
        'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
        'AppEn', 'SampEn',"Trans_AppEn"]))

In [11]:
export_df

,Mean Acc.,SD of Acc.,Total Acc.,Sensitivity,Specificity,Precision,f1
StoS,0.692705,0.266921,0.667052,0.726873,0.618155,0.608762,0.662595
StoR,0.898461,0.112915,0.891367,0.934289,0.856283,0.841618,0.885536
StoL,0.721097,0.159525,0.708993,0.583189,0.811826,0.716978,0.643200
RtoS,0.845485,0.132266,0.836395,0.841780,0.831993,0.803748,0.822325
RtoR,0.883605,0.139107,0.875200,0.901607,0.853615,0.834286,0.866641
RtoL,0.907903,0.077323,0.901530,0.859679,0.935739,0.916214,0.887047
LtoS,0.785057,0.178731,0.769147,0.542744,0.954209,0.906441,0.678955
LtoR,0.822755,0.118014,0.820361,0.856712,0.790648,0.769849,0.810961
LtoL,0.692346,0.243333,0.670677,0.306947,0.967991,0.886857,0.456051
STD,0.786712,0.194953,0.784780,0.871545,0.713858,0.713442,0.784608


In [12]:
export_df_highlight = export_df.copy()

export_df_highlight.style.highlight_max(color = 'lightgreen', axis = 0)

,Mean Acc.,SD of Acc.,Total Acc.,Sensitivity,Specificity,Precision,f1
StoS,0.692705,0.266921,0.667052,0.726873,0.618155,0.608762,0.662595
StoR,0.898461,0.112915,0.891367,0.934289,0.856283,0.841618,0.885536
StoL,0.721097,0.159525,0.708993,0.583189,0.811826,0.716978,0.643200
RtoS,0.845485,0.132266,0.836395,0.841780,0.831993,0.803748,0.822325
RtoR,0.883605,0.139107,0.875200,0.901607,0.853615,0.834286,0.866641
RtoL,0.907903,0.077323,0.901530,0.859679,0.935739,0.916214,0.887047
LtoS,0.785057,0.178731,0.769147,0.542744,0.954209,0.906441,0.678955
LtoR,0.822755,0.118014,0.820361,0.856712,0.790648,0.769849,0.810961
LtoL,0.692346,0.243333,0.670677,0.306947,0.967991,0.886857,0.456051
STD,0.786712,0.194953,0.784780,0.871545,0.713858,0.713442,0.784608


In [13]:
export_df_highlight.style.highlight_min(color = 'lightcoral', axis = 0)

,Mean Acc.,SD of Acc.,Total Acc.,Sensitivity,Specificity,Precision,f1
StoS,0.692705,0.266921,0.667052,0.726873,0.618155,0.608762,0.662595
StoR,0.898461,0.112915,0.891367,0.934289,0.856283,0.841618,0.885536
StoL,0.721097,0.159525,0.708993,0.583189,0.811826,0.716978,0.643200
RtoS,0.845485,0.132266,0.836395,0.841780,0.831993,0.803748,0.822325
RtoR,0.883605,0.139107,0.875200,0.901607,0.853615,0.834286,0.866641
RtoL,0.907903,0.077323,0.901530,0.859679,0.935739,0.916214,0.887047
LtoS,0.785057,0.178731,0.769147,0.542744,0.954209,0.906441,0.678955
LtoR,0.822755,0.118014,0.820361,0.856712,0.790648,0.769849,0.810961
LtoL,0.692346,0.243333,0.670677,0.306947,0.967991,0.886857,0.456051
STD,0.786712,0.194953,0.784780,0.871545,0.713858,0.713442,0.784608


In [14]:
export_df.to_csv(f'individual-features/XGBoost.csv')